In [3]:
# %%
import tkinter as tk
from tkinter import filedialog, messagebox
import win32com.client as win32
import pandas as pd
import logging
import os

# 로그 설정 (디버깅 수준으로 설정)
logging.basicConfig(level=logging.DEBUG, format='%(asctime)s - %(levelname)s - %(message)s')

# 한글 파일 및 엑셀 파일 경로를 저장할 변수
hwp_file_path = ""
excel_file_path = ""

# Tkinter 창 설정
root = tk.Tk()
root.title("한글 및 엑셀 파일 열기 도구")
root.geometry("400x300")

# 한글 파일 경로 라벨
hwp_file_label = tk.Label(root, text="한글 파일: 아직 선택되지 않음")
hwp_file_label.pack(pady=5)

# 엑셀 파일 경로 라벨
excel_file_label = tk.Label(root, text="엑셀 파일: 아직 선택되지 않음")
excel_file_label.pack(pady=5)

def open_hwp():
    global hwp_file_path
    # 한글 파일 선택하기
    hwp_file_path = filedialog.askopenfilename(
        title="한글 파일 선택",
        filetypes=[("HWP files", "*.hwp"), ("All files", "*.*")]
    )
    
    # 파일이 선택되지 않았으면 아무것도 하지 않음
    if not hwp_file_path:
        return

    # 선택한 파일의 이름만 라벨에 표시
    hwp_file_label.config(text=f"한글 파일: {os.path.basename(hwp_file_path)}")

def open_excel():
    global excel_file_path
    # 엑셀 파일 선택하기
    excel_file_path = filedialog.askopenfilename(
        title="엑셀 파일 선택",
        filetypes=[("Excel files", "*.xlsx;*.xls"), ("All files", "*.*")]
    )
    
    # 파일이 선택되지 않았으면 아무것도 하지 않음
    if not excel_file_path:
        return

    # 선택한 파일의 이름만 라벨에 표시
    excel_file_label.config(text=f"엑셀 파일: {os.path.basename(excel_file_path)}")

def process_files():
    if not hwp_file_path or not excel_file_path:
        messagebox.showerror("오류", "먼저 한글 파일과 엑셀 파일을 모두 선택하세요.")
        return

    # 엑셀 파일 데이터 읽기
    try:
        excel_data = pd.read_excel(excel_file_path, sheet_name="참여자 명단", header=1, dtype=str).fillna("")
    except Exception as e:
        messagebox.showerror("오류", f"엑셀 파일을 여는 도중 오류가 발생했습니다: {e}")
        return

    # 한글 파일 처리
    try:
        hwp = win32.gencache.EnsureDispatch("HWPFrame.HwpObject")
        hwp.RegisterModule("FilePathCheckDLL", "FilePathCheckerModuleExample")
        hwp.XHwpWindows.Item(0).Visible = True  # 한글 창을 보이게 설정

        for index, row in excel_data.iterrows():
            hwp.Open(hwp_file_path)

            if not row['성명']:
                logging.debug(f"빈 값 발견 - 행 {index + 1} 건너뜀")
                continue

            try:
                # 필드에 값을 삽입
                hwp.PutFieldText('성명', row['성명'])
                hwp.PutFieldText('주민등록번호', row['주민등록번호'])
                hwp.PutFieldText('만나이', row['만나이'])
                hwp.PutFieldText('주소', row['주소'])
                hwp.PutFieldText('연락처', row['연락처'])
                hwp.PutFieldText('이메일', row['이메일'])
                hwp.PutFieldText('최종학과', row['최종학과'])
                hwp.PutFieldText('작성날짜', row['작성날짜'])
                hwp.PutFieldText('최종학교명', row['최종학교명'])

                # 최종학력구분에 따라 기호를 추가
                if row['최종학력구분'] == '대학교_4년제':
                    hwp.PutFieldText('대학교', '■')
                    hwp.PutFieldText('대학교명', row['최종학교명'])
                    if row['최종학력상태'] == '졸업':
                        hwp.PutFieldText('대학교졸업', '■')
                    elif row['최종학력상태'] == '졸업예정자':
                        hwp.PutFieldText('졸업예정자', '■')
                    elif row['최종학력상태'] == '휴학':
                        if row['학년'] in ['1','2']:
                            hwp.PutFieldText('재학(1~2학년)', '■')
                        elif row['학년'] in ['3','4']:
                            hwp.PutFieldText('재학(3~4학년)', '■')       
                    elif row['최종학력상태'] == '재학':
                        if row['학년'] in ['1','2']:
                            hwp.PutFieldText('재학(1~2학년)', '■')
                        elif row['학년'] in ['3','4']:
                            hwp.PutFieldText('재학(3~4학년)', '■')  

                elif row['최종학력구분'] == '고등학교':
                    hwp.PutFieldText('고등학교', '■')
                    hwp.PutFieldText('고등학교명', row['최종학교명'])
                    if row['최종학력상태'] == '졸업':
                        hwp.PutFieldText('고등학교졸업', '■')
                    elif row['최종학력상태'] == '졸업예정자':
                        hwp.PutFieldText('고등학교졸업예정자', '■')
                    elif row['최종학력상태'] == '재학':
                        if row['학년'] in ['1', '2']:
                            hwp.PutFieldText('고등학교재학(1~2학년)', '■')
                        elif row['학년'] == '3':
                            hwp.PutFieldText('고등학교재학(3학년)', '■')
                    elif row['최종학력상태'] == '검정고시':
                        hwp.PutFieldText('검정고시', '■')
                
                # 파일을 새로운 이름으로 저장
                new_file_name = f'{os.path.splitext(os.path.basename(hwp_file_path))[0]}_{row["성명"]}.hwp'
                new_file_path = os.path.join(os.path.dirname(hwp_file_path), new_file_name)
                hwp.SaveAs(new_file_path)

                logging.info(f'파일 저장 완료: {new_file_name}')
                hwp.Clear(1)
            except Exception as e:
                logging.error(f"오류 발생 (행 {index + 1}): {e}")

        messagebox.showinfo("완료", "엑셀 데이터로 한글 파일이 성공적으로 처리되었습니다.")
    except Exception as e:
        messagebox.showerror("오류", f"한글 파일을 처리하는 도중 오류가 발생했습니다: {e}")
    finally:
        hwp.Quit()

# 한글 파일 열기 버튼
open_hwp_button = tk.Button(root, text="한글 파일 열기", command=open_hwp)
open_hwp_button.pack(pady=10)

# 엑셀 파일 열기 버튼
open_excel_button = tk.Button(root, text="엑셀 파일 열기", command=open_excel)
open_excel_button.pack(pady=10)

# 한글 및 엑셀 파일 처리 버튼
process_button = tk.Button(root, text="한글 및 엑셀 파일 처리", command=process_files)
process_button.pack(pady=20)

# Tkinter 창 실행
root.mainloop()
